In [1]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd
from IPython.display import display
import sqlite3
import json
import numpy as np
import re
from openpyxl import load_workbook
import xlsxwriter
import requests
from requests.auth import HTTPBasicAuth

In [2]:
#FUNCTION TO IMPORT FOLDERS
def import_csv(path):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            file_path = os.path.join(path, file)
            file_data = pd.read_csv(file_path)
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw
    
def import_xlsx(path, sheet_name):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".xlsx"):
            file_path = os.path.join(path, file)
            file_data = pd.read_excel(file_path, engine="openpyxl", sheet_name = sheet_name)
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw

In [3]:
# source
personal_path = os.environ['USERPROFILE']
if personal_path in ['C:\\Users\\dinhhoang.nguyen.CONCENTRIX', 'C:\\Users\\ADMIN']:
    middle_path = r'OneDrive - Concentrix Corporation\WFM-internal'
else:
    middle_path = r'Concentrix Corporation\Dinh Hoang Nguyen - WFM-internal'
user_credential = os.path.join(os.environ['USERPROFILE'], middle_path)
# Adherence file path
adherence_path=os.path.join(user_credential, r"Dispatch files\Adherence & Daily Tracking\Adherence.xlsx")
# Revenue Estimation file path
revenue_path=os.path.join(user_credential, r"Dispatch files\Adherence & Daily Tracking\Revenue Estimation.xlsx")
# Daily Tracking file path
dailytracking_path=os.path.join(user_credential, r"Dispatch files\Adherence & Daily Tracking\Daily Tracking.xlsx")
# EPS path
eps_path = os.path.join(user_credential, r"Data\EPS Tableau\EPS Query")
# Budget path
budget_path = os.path.join(user_credential, r"Data\Budget")
# Define billable
define_path=os.path.join(user_credential, r"Data\Define.xlsx")

In [4]:
# IMPORT DEFINE
define = pd.read_excel(define_path, sheet_name="DEFINE")

In [5]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(user_credential, r'DB\filejson\masterroster.json'))
masterroster_daily=masterroster[['Employee_ID','TED Name', 'PST_Start_Date', 'Role','Booking Login ID','Full name','Wave #']]
masterroster_daily['Employee_ID'] = masterroster_daily['Employee_ID'].astype("Int64")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\242608621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masterroster_daily['Employee_ID'] = masterroster_daily['Employee_ID'].astype("Int64")


In [6]:
# IMPORT OT
ot = pd.read_json(os.path.join(user_credential, r'DB\filejson\ot.json'))
ot_daily=ot[['Emp ID','Date','OT']]
ot_daily=ot_daily.groupby(['Emp ID','Date'], as_index=False).sum()
ot_daily['Emp ID']=ot_daily['Emp ID'].astype("Int64")
ot_daily['Date'] = pd.to_datetime(ot_daily['Date'], format="mixed").dt.date

In [7]:
# IMPORT RAMCO
ramco_daily = pd.read_json(os.path.join(user_credential, r'DB\filejson\ramco.json'))
ramco_daily['Date']=pd.to_datetime(ramco_daily['Date'],format='mixed').dt.date
ramco_daily['EID'] = ramco_daily['EID'].astype("Int64")
ramco_daily['Value']=ramco_daily['Value'].fillna(0)
ramco_daily=ramco_daily.rename(columns={'Value':'RAMCO'})
ramco_ot_daily=pd.merge(ramco_daily,ot_daily,left_on=['EID','Date'],right_on=['Emp ID','Date'],how='left')
def OTREGISTERED_func(x):
    if x['RAMCO'] =='PO' or x['RAMCO'] =='PH' :
        return x['OT']/24
    elif x['RAMCO'] =='WO' or x['RAMCO'] =='AL' or x['RAMCO'] =='SL' or x['RAMCO'] =='HO':
        return 0 
    elif x['RAMCO'] =='HAL' :
        return 3.75/24
    else:
        return x['OT']/24+7.5/24
ramco_ot_daily['OT Registered'] = ramco_ot_daily.apply(OTREGISTERED_func, axis=1)
ramco_ot_daily=ramco_ot_daily[['EID','Date','RAMCO','OT','OT Registered']]

In [8]:
# IMPORT RAMCOOT
ramcoot = pd.read_json(os.path.join(user_credential, r'DB\filejson\ramcoot.json'))

In [9]:
# IMPORT SCHEDULE
schedule = pd.read_json(os.path.join(user_credential, r'DB\filejson\schedule.json'))
schedule['Date']=pd.to_datetime(schedule['Date']).dt.date

In [10]:
#IMPORT CUIC
cuic = pd.read_json(os.path.join(user_credential, r'DB\filejson\cuic.json'))

In [11]:
#IMPORT CPI
cpi = pd.read_json(os.path.join(user_credential, r'DB\filejson\cpi.json'))
# Clean
cpi_daily=cpi[['Staff Name', 'Employee_ID', 'Date', 'Channel', 'Number of Records']]
condition_phone=(cpi['Channel']=='phone')
cpi_daily['Phone Cases #']= np.where(condition_phone, cpi['Number of Records'], 0)
condition_nonphone=(cpi['Channel']=='email')|(cpi['Channel']=='messaging')|(cpi['Channel']=='Live Chat')
cpi_daily['Non-phone Cases #']= np.where(condition_nonphone, cpi['Number of Records'], 0)
cpi_daily=cpi_daily.rename(columns={'Number of Records':'Cases #'})
cpi_daily=cpi_daily[['Date','Employee_ID','Cases #','Phone Cases #','Non-phone Cases #']]
cpi_daily['Date']=pd.to_datetime(cpi_daily['Date'],format='mixed').dt.date
cpi_daily=cpi_daily.groupby(['Employee_ID','Date'], as_index=False).sum()
cpi_daily['Employee_ID'] = cpi_daily['Employee_ID'].astype("Int64")

In [12]:
# IMPORT TL
tl = pd.read_json(os.path.join(user_credential, r'DB\filejson\tl.json'))

In [13]:
# IMPORT EXCEPTION
exception = pd.read_json(os.path.join(user_credential, r'DB\filejson\exceptional_hrs.json'))
exception['Date']=pd.to_datetime(exception['Date'],format='mixed').dt.date
exception_daily=exception[['Emp ID','Date','Exception request \n(Minute)']]
exception_daily['Exception request \n(Minute)'] = exception_daily['Exception request \n(Minute)'].astype("float")
exception_daily['Exception request \n(Minute)']=exception['Exception request \n(Minute)'].fillna(0)
exception_daily=exception_daily.rename(columns={'Exception request \n(Minute)':'Exception request'})
exception_daily=exception_daily.groupby(['Emp ID','Date'], as_index=False).sum()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\508952134.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exception_daily['Exception request \n(Minute)'] = exception_daily['Exception request \n(Minute)'].astype("float")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\508952134.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exception_daily['Exception request \n(Minute)']=exception['Exception request \n(Minute)'].fillna(0)


In [14]:
#IMPORT EPS
eps_raw = import_csv(path = eps_path)
eps = eps_raw.drop(columns=['Index', 'filename', 'Date'])
eps['Session Login'] = pd.to_datetime(eps['Session Login'], errors='coerce')
eps['Session Login'] = eps['Session Login'].dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC')
eps['Session Logout'] = pd.to_datetime(eps['Session Logout'], errors='coerce')
eps['Session Logout'] = eps['Session Logout'].dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC')
eps['Session Login'] =pd.to_datetime(eps['Session Login'], errors='coerce').dt.tz_localize(None)
eps['Session Logout'] =pd.to_datetime(eps['Session Logout'], errors='coerce').dt.tz_localize(None)

eps['Session login VN'] = pd.to_datetime(eps['Session Login'], format ='mixed') + pd.Timedelta(hours=7)
eps['Session logout VN'] = pd.to_datetime(eps['Session Logout'], format ='mixed') + pd.Timedelta(hours=7)
eps['Session login date'] = pd.to_datetime(eps['Session login VN'], format ='mixed').dt.date
eps['Session login time'] = pd.to_datetime(eps['Session login VN'], format ='mixed').dt.time
eps['Session login hour'] = pd.to_datetime(eps['Session login VN'], format ='mixed').dt.strftime('%H')
eps['Session logout date'] = pd.to_datetime(eps['Session logout VN'], format ='mixed').dt.date
eps['Session logout time'] = pd.to_datetime(eps['Session logout VN'], format ='mixed').dt.time
eps['Session logout hour'] = pd.to_datetime(eps['Session logout VN'], format ='mixed').dt.strftime('%H')
eps['Session login hour']=eps['Session login hour'].astype("Int64")
eps['Session logout hour']=eps['Session logout hour'].astype("Int64")
nighthour_23h="23:00:00"
nighthour_7h="07:00:00"
nighthour_24h="00:00:00"
eps['Session login 23h'] = pd.to_datetime(eps['Session login date'].astype(str) + nighthour_23h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session logout 23h'] = pd.to_datetime(eps['Session logout date'].astype(str) + nighthour_23h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session login 7h'] = pd.to_datetime(eps['Session login date'].astype(str) + nighthour_7h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session logout 7h'] = pd.to_datetime(eps['Session logout date'].astype(str) + nighthour_7h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session login 24h'] = pd.to_datetime(eps['Session login date'].astype(str) + nighthour_24h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')+pd.Timedelta(days=1)
eps['Session logout 24h'] = pd.to_datetime(eps['Session logout date'].astype(str) + nighthour_24h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')+pd.Timedelta(days=0)
eps=eps[eps['EID'].isnull() == False]
def night_func(x):
    if (x['Session login hour']>=23 and x['Session logout hour']>=23) or (x['Session login hour']>=23 and x['Session logout hour']<7) or (x['Session login hour']<7 and x['Session logout hour']<7):
        return x['Total Time']
    elif x['Session login hour']>=23 and x['Session logout hour']>=7:
        if ((x['Session login 24h']-x['Session login VN'])+pd.Timedelta(hours=7)).total_seconds()<x['Total Time']:
            return ((x['Session login 24h']-x['Session login VN'])+pd.Timedelta(hours=7)).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<7 and x['Session logout hour']>=7 and x['Session logout hour']<23:
        if (x['Session login 7h']-x['Session login VN']).total_seconds()<x['Total Time']:
            return (x['Session login 7h']-x['Session login VN']).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<7 and x['Session logout hour']>=23:
        if ((x['Session login 7h']-x['Session login VN'])+(x['Session logout VN']-x['Session logout 23h'])).total_seconds()<x['Total Time']:
            return ((x['Session login 7h']-x['Session login VN'])+(x['Session logout VN']-x['Session logout 23h'])).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<23 and x['Session logout hour']>=23:
        if (x['Session logout VN']-x['Session logout 23h']).total_seconds()<x['Total Time']:
            return (x['Session logout VN']-x['Session logout 23h']).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<23 and x['Session logout hour']<7:
        if (x['Session logout VN']-x['Session login 23h']).total_seconds()<x['Total Time']:
            return (x['Session logout VN']-x['Session login 23h']).total_seconds()
        else :
            return x['Total Time']
    else:
        return (x['Session login VN']-x['Session login VN']).total_seconds()
eps['night']= eps.apply(night_func, axis=1)

eps = eps.drop(columns=['Session login VN', 'Session logout VN', 'Username', 'manager_username', 'sitecode', 'Session Time',
                        'Session login hour','Session logout hour','Session login 23h','Session login 7h','Session login 24h'
                       ,'Session logout 23h','Session logout 7h','Session logout 24h'])
eps['EID']=eps['EID'].astype("Int64")
# 1. Merge EPS and Schedule on EID and Date to get Shift_type, Shift, Session Date and Duration and change Date into VN Date
eps_schedule=pd.merge(eps, schedule, left_on=['EID','Session login date'], right_on=['Emp ID','Date'], how='left')
eps_schedule=eps_schedule[['EID','Session Login','Session Logout','BPE Code','Total Time','Session login date','Session login time',
                           'Session logout date','Session logout time','LOB','Shift_type','Shift','night']]
eps_schedule['Date-1']=pd.to_datetime(eps_schedule['Session login date'], format ='mixed') - pd.Timedelta(1,"d")
eps_schedule['Date-1']=pd.to_datetime(eps_schedule['Date-1'], format ='mixed').dt.date
eps_schedule = pd.merge(eps_schedule, schedule, left_on=['EID','Date-1'], right_on =['Emp ID','Date'], how='left')
mytime=t(12,0,0)
condition=(eps_schedule['Shift_type_x']!='DS')&(eps_schedule['Shift_type_y']=='NS')&(eps_schedule['Session login time']<mytime)
eps_schedule=eps_schedule.rename(columns={'LOB_x': 'LOB','Shift_type_x':'Shift_type','Shift_x':'Shift'})
eps_schedule['Session Date'] = np.where(condition, eps_schedule['Date-1'], eps_schedule['Session login date'])
eps_schedule['Duration']=eps_schedule['Total Time']/86400
eps_schedule['night']=eps_schedule['night']/86400
eps_schedule=eps_schedule[['EID','Session Login','Session Logout','BPE Code','Total Time','Session login date','Session login time',
                           'Session logout date','Session logout time','LOB','Session Date','Duration','night']]
eps_schedule = pd.merge(eps_schedule, schedule, left_on=['EID','Session Date'], right_on =['Emp ID','Date'], how='left')
eps_schedule=eps_schedule.rename(columns={'LOB_y': 'LOB'})
eps_schedule=eps_schedule[['EID','Session Login','Session Logout','BPE Code','Total Time','Session login date','Session login time',
                           'Session logout date','Session logout time','LOB','Shift_type','Session Date','Duration','Shift','night']]

# 2. Add Week & Day Name and BPE Code Duration and then group by EID, Session Date
eps_dailytracking=eps_schedule
eps_dailytracking['Session Date'] = pd.to_datetime(eps_dailytracking['Session Date'],errors ='coerce')
# Add Week & Day Name column
eps_dailytracking['Week'] = eps_dailytracking['Session Date'].dt.strftime('%Y%W')
eps_dailytracking['Day Name'] = eps_dailytracking['Session Date'].dt.day_name()

# Add Picklist,Ready or Talking,..BPE Code
condition_picklist=(eps_dailytracking['BPE Code']=='Picklist - off Phone')
eps_dailytracking['Picklist - off Phone']= np.where(condition_picklist, eps_dailytracking['Duration'], 0)

condition_readyortalking=(eps_dailytracking['BPE Code']=='Ready or Talking')
eps_dailytracking['Ready or Talking']= np.where(condition_readyortalking, eps_dailytracking['Duration'], 0)

condition_finalizecall=(eps_dailytracking['BPE Code']=='Finalize Call')
eps_dailytracking['Finalize Call']= np.where(condition_finalizecall, eps_dailytracking['Duration'], 0)

condition_meeting=(eps_dailytracking['BPE Code']=='Meeting')
eps_dailytracking['Meeting']= np.where(condition_meeting, eps_dailytracking['Duration'], 0)

condition_training=(eps_dailytracking['BPE Code']=='Training')
eps_dailytracking['Training']= np.where(condition_training, eps_dailytracking['Duration'], 0)

condition_newhiretraining=(eps_dailytracking['BPE Code']=='New Hire Training')
eps_dailytracking['New Hire Training']= np.where(condition_newhiretraining, eps_dailytracking['Duration'], 0)

condition_lunch=(eps_dailytracking['BPE Code']=='Lunch')
eps_dailytracking['Lunch Time']= np.where(condition_lunch, eps_dailytracking['Duration'], 0)

condition_break=(eps_dailytracking['BPE Code']=='Break')
eps_dailytracking['Break Time']= np.where(condition_break, eps_dailytracking['Duration'], 0)

condition_rona=(eps_dailytracking['BPE Code']=='RONA')
eps_dailytracking['RONA']= np.where(condition_rona, eps_dailytracking['Duration'], 0)

condition_othercodecph=(eps_dailytracking['BPE Code']=='Unscheduled Picklist')|(eps_dailytracking['BPE Code']=='Payment Processing')|(eps_dailytracking['BPE Code']=='Social Media')|(eps_dailytracking['BPE Code']=='Mass Issue')|(eps_dailytracking['BPE Code']=='Project')
eps_dailytracking['Other code CPH']= np.where(condition_othercodecph, eps_dailytracking['Duration'], 0)

condition_others=(eps_dailytracking['BPE Code']!='Picklist - off Phone') & (eps_dailytracking['BPE Code']!='Ready or Talking') & (eps_dailytracking['BPE Code']!='Meeting')& (eps_dailytracking['BPE Code']!='Training') & (eps_dailytracking['BPE Code']!='Lunch') & (eps_dailytracking['BPE Code']!='Break')& (eps_dailytracking['BPE Code']!='New Hire Training')& (eps_dailytracking['BPE Code']!='RONA')
eps_dailytracking['Other codes']= np.where(condition_others, eps_dailytracking['Duration'], 0)

condition_nightproductivehours_x=(eps_dailytracking['BPE Code']=='Picklist - off Phone')
eps_dailytracking['Night Picklist - off Phone']= np.where(condition_nightproductivehours_x, eps_dailytracking['night'], 0)

condition_nightproductivehours_y=(eps_dailytracking['BPE Code']=='Ready or Talking')
eps_dailytracking['Night Ready or Talking']= np.where(condition_nightproductivehours_y, eps_dailytracking['night'], 0)

condition_nightproductivehours_z=(eps_dailytracking['BPE Code']=='Finalize Call')
eps_dailytracking['Night Finalize Call']= np.where(condition_nightproductivehours_z, eps_dailytracking['night'], 0)

condition_nightdowntimehours_x=(eps_dailytracking['BPE Code']=='Training')
eps_dailytracking['Night Training']= np.where(condition_nightdowntimehours_x, eps_dailytracking['night'], 0)

condition_nightdowntimehours_y=(eps_dailytracking['BPE Code']=='Meeting')
eps_dailytracking['Night Meeting']= np.where(condition_nightdowntimehours_y, eps_dailytracking['night'], 0)

eps_dailytracking['Night Productive Hours']=eps_dailytracking['Night Picklist - off Phone']+eps_dailytracking['Night Ready or Talking']+eps_dailytracking['Night Finalize Call']
eps_dailytracking['Night Downtime Hours']=eps_dailytracking['Night Training']+eps_dailytracking['Night Meeting']
eps_dailytracking=eps_dailytracking[['EID','LOB','Shift_type','Session Date','Duration','Shift','Week','Day Name',
                                     'Picklist - off Phone','Ready or Talking','Meeting','Training','New Hire Training','Lunch Time','Break Time',
                                     'Other codes','Night Productive Hours','Night Downtime Hours','Finalize Call','Other code CPH','RONA']]
eps_dailytracking=eps_dailytracking.groupby(['EID','LOB','Shift_type','Session Date','Shift','Week','Day Name'], as_index=False).sum()
eps_dailytracking['Session Date']=pd.to_datetime(eps_dailytracking['Session Date'],format='mixed').dt.date

In [15]:
# Login_Logout Time
login_logout=eps_schedule[['EID','Session Date','Shift_type','Session login time','Session logout time']]

# Login_Logout Time: filter !=NS
login_logout_ds=login_logout.loc[(login_logout['Shift_type'] !='NS')]
login_logout_ds=login_logout_ds.groupby(['EID','Session Date','Shift_type'], as_index=False).agg(min=('Session login time', 'min'), max=('Session logout time', 'max'))
login_logout_ds=login_logout_ds.rename(columns={'min':'Login Time','max':'Logout Time'})

# Login_Logout Time: filter NS (condition: login sau 15h & logout >15h)
validate_time=t(15,0,0)
login_ns=login_logout.loc[(login_logout['Shift_type'] =='NS')&(login_logout['Session login time'] > validate_time)]
login_ns=login_ns.groupby(['EID','Session Date','Shift_type'], as_index=False).agg(min=('Session login time', 'min'))
logout_ns=login_logout.loc[(login_logout['Shift_type'] =='NS')&(login_logout['Session logout time'] < validate_time)]
logout_ns=logout_ns.groupby(['EID','Session Date','Shift_type'], as_index=False).agg(max=('Session logout time', 'max'))
login_logout_ns=pd.merge(login_ns,logout_ns,left_on=['EID','Session Date','Shift_type'],right_on=['EID','Session Date','Shift_type'],how='left')
login_logout_ns=login_logout_ns.rename(columns={'min':'Login Time','max':'Logout Time'})

# Login_Logout Time:Combine DS & NS login logout time
login_logout=pd.concat([login_logout_ds, login_logout_ns], ignore_index=True, sort=False)

In [16]:
### Daily tracking
# Merge CPI to get Case #
dailytracking=pd.merge(eps_dailytracking,cpi_daily,left_on=['EID','Session Date'],right_on=['Employee_ID','Date'],how='left')
dailytracking=dailytracking.drop(columns=['Employee_ID','Date'])

# Merge Exception to get Exception minute
dailytracking=pd.merge(dailytracking,exception_daily,left_on=['EID','Session Date'],right_on=['Emp ID','Date'],how='left')
dailytracking=dailytracking.drop(columns=['Emp ID','Date'])

# Merge Ramco & OT to get Ramco, OT, OT Registered
dailytracking=pd.merge(dailytracking,ramco_ot_daily,left_on=['EID','Session Date'],right_on=['EID','Date'],how='left')
dailytracking=dailytracking.drop(columns=['Date'])

# Merge Master Roster to get Full name, Booking Login ID, TED Name, Wave #, Employee_ID
dailytracking=pd.merge(dailytracking,masterroster_daily,left_on=['EID'],right_on=['Employee_ID'],how='left')
dailytracking=dailytracking.drop(columns=['Employee_ID'])

# Merge TL to get Supervisor
dailytracking=pd.merge(dailytracking,tl,left_on=['EID'],right_on=['Emp ID'],how='left')
dailytracking=dailytracking.drop(columns=['Emp ID'])

# Merge Login Logout Time
login_logout['Session Date']=pd.to_datetime(login_logout['Session Date']).dt.date
dailytracking=pd.merge(dailytracking,login_logout,left_on=['EID','Session Date','Shift_type'],right_on=['EID','Session Date','Shift_type'],how='left')

dailytracking['Cases #']=dailytracking['Cases #'].fillna(0)
dailytracking['Phone Cases #']=dailytracking['Phone Cases #'].fillna(0)
dailytracking['Non-phone Cases #']=dailytracking['Non-phone Cases #'].fillna(0)
dailytracking['RAMCO']=dailytracking['RAMCO'].fillna(0)
dailytracking['OT']=dailytracking['OT'].fillna(0)
dailytracking['Exception request']=dailytracking['Exception request'].fillna(0)

dailytracking['Productive hours']=dailytracking['Picklist - off Phone']+dailytracking['Ready or Talking']+dailytracking['Finalize Call']
dailytracking['Downtime']=dailytracking['Meeting']+dailytracking['Training']
dailytracking['Delivery hours']=dailytracking['Productive hours']+dailytracking['Downtime']+dailytracking['New Hire Training']
dailytracking=dailytracking.rename(columns={'Duration':'Staffed Hours'})
dailytracking['CPH - Agent']=dailytracking['Cases #']/((dailytracking['Productive hours']+dailytracking['RONA']+dailytracking['Other code CPH'])*24)
dailytracking['Phone CPH']=dailytracking['Phone Cases #']/((dailytracking['Ready or Talking']+dailytracking['Finalize Call'])*24)
dailytracking['Non-phone CPH']=dailytracking['Non-phone Cases #']/(dailytracking['Picklist - off Phone']*24)
dailytracking['Picklist %']=(dailytracking['Picklist - off Phone']/dailytracking['Delivery hours']).mul(100).round(2).astype(str).add('%')
def standardtime_func(x):
    if x['RAMCO'] =='PR' or x['RAMCO'] =='PH' :
        return x['OT']+8
    elif x['RAMCO'] =='PO':
        return x['OT']
    else:
        return 0
dailytracking['Standard time']= dailytracking.apply(standardtime_func, axis=1)
dailytracking['Extra rendered hours']=(dailytracking['Delivery hours']+dailytracking['Break Time']+(dailytracking['Exception request']/(60*24)))*24-dailytracking['Standard time']
def approvedovertime_func(x):
    if x['OT']<=0:
        return 0
    elif x['RAMCO'] not in ['PR','PO','PH']:
        return 0
    elif x['Extra rendered hours'] >= 0:
        return  x['OT']
    elif (x['OT']+x['Extra rendered hours']-0.25) < 0 :
        return  0
    else:         
        return x['OT']+x['Extra rendered hours']-0.25
dailytracking['Approved Overtime']= dailytracking.apply(approvedovertime_func, axis=1)
dailytracking['Approved Overtime']=dailytracking['Approved Overtime'].round(0)
def convert_to_hours(delta):
    total_seconds = delta.total_seconds()
    hours = str(int(total_seconds // 3600)).zfill(2)
    minutes = str(int((total_seconds % 3600) // 60)).zfill(2)
    seconds = str(int(total_seconds % 60)).zfill(2)
    return f"{hours}:{minutes}:{seconds}"
validate_date=dt(1970,1,1,0,0,0)
dailytracking['Day Productive Hours']=dailytracking['Productive hours']-dailytracking['Night Productive Hours']
dailytracking['Day Downtime Hours']=dailytracking['Downtime']-dailytracking['Night Downtime Hours']
dailytracking=dailytracking[['Week','Session Date','Day Name','EID','Full name','Booking Login ID','TED Name','Wave #','RAMCO','LOB','Shift','Shift_type',
                             'Supervisor','Login Time','Logout Time','Picklist - off Phone','Ready or Talking','Productive hours','Meeting','Training',
                             'Downtime','Delivery hours','Lunch Time','Break Time','RONA','Other codes','Staffed Hours','Exception request','Cases #',
                             'CPH - Agent','Phone Cases #','Phone CPH','Non-phone Cases #','Non-phone CPH','Picklist %','OT','Approved Overtime','New Hire Training',
                             'Night Productive Hours','Night Downtime Hours','Day Productive Hours','Day Downtime Hours']]
dailytracking['Session Date'] = pd.to_datetime(dailytracking['Session Date'])
dailytracking=dailytracking.sort_values(by='Session Date',ascending=True)
#ATD MM
def atdmm_shift_func(x):
    if x['Shift'] in ['0400-1300','0500-1400','0600-1500','0700-1600','0800-1700','0900-1800','1000-1900','1100-2000',
                      '1200-2100','1300-2200','1400-2300','1500-2400','1600-0100','1700-0200',
                      '1800-0300','1900-0400','2000-0500','2100-0600','2200-0700','HAL','Training','PEGA']:
        return 'WORK'
    else:         
        return 'OFF'
dailytracking['ATD MM Shift']= dailytracking.apply(atdmm_shift_func, axis=1)
def atdmm_ramco_func(x):
    if x['RAMCO'] in ['PH','PO','PR','PI','POWH','HAL','HLWP','HSL']:
        return 'WORK'
    else:         
        return 'OFF'
dailytracking['ATD MM Ramco']= dailytracking.apply(atdmm_ramco_func, axis=1)
def atdmm_func(x):
    if x['EID'] =='':
        return ''
    elif x['RAMCO']=='PO' and x['Shift']=='OFF':
        return 'Valid'   
    elif x['RAMCO']=='' and x['Shift']!='':
        return '' 
    elif x['RAMCO']!='' and x['Shift']=='':
        return ''     
    elif x['RAMCO']=='' and x['Shift']=='':
        return 'Valid'  
    elif x['RAMCO']=='AB' and x['ATD MM Shift']=='WORK':
        return 'Valid' 
    elif x['ATD MM Ramco'] == x['ATD MM Shift']:
        return 'Valid' 
    else:         
        return 'ATD MM'
dailytracking['ATD MM']= dailytracking.apply(atdmm_func, axis=1)
#PO (MTD)
def po_func(x):
    if x['RAMCO'] in ['PO']:
        return 1
    else:         
        return 0
dailytracking['PO']= dailytracking.apply(po_func, axis=1)
dailytracking['Session Date'] = pd.to_datetime(dailytracking['Session Date'])
dailytracking['#PO (MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['PO'].cumsum()
#POHrs (MTD)
def pohour_func(x):
    if x['RAMCO'] in ['PO']:
        return x['Approved Overtime']
    else:         
        return 0
dailytracking['PO Hour']= dailytracking.apply(pohour_func, axis=1)
dailytracking['POHrs(MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['PO Hour'].cumsum()
#OT (MTD)
dailytracking['OT (MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['Approved Overtime'].cumsum()
# Final OT
def finalOT_func(x):
    if x['OT (MTD)'] <= 40 :
        return x['Approved Overtime']
    else:         
        return 0
dailytracking['Final OT']= dailytracking.apply(finalOT_func, axis=1)
#OT OTP
def OT_OTP_func(x):
    if x['OT (MTD)'] > 40 :
        return x['Approved Overtime']
    else:         
        return 0
dailytracking['OT OTP']= dailytracking.apply(OT_OTP_func, axis=1)
#OT-LessDelivery
def OT_LessDelivery_func(x):
    if x['Approved Overtime'] > x['OT'] :
        return 1
    else:         
        return 0
dailytracking['OT-LessDelivery']= dailytracking.apply(OT_LessDelivery_func, axis=1)
#OT>4Hrs On PR & OT>8Hrs On PO
def OT_on_PO_func(x):
    if x['RAMCO'] =='PR' and x['Approved Overtime']>4 :
        return 1
    elif x['RAMCO'] =='PO' and x['Approved Overtime']>8 :
        return 1 
    else:         
        return 0
dailytracking['OT>4Hrs On PR \n OT>8Hrs On PO']= dailytracking.apply(OT_on_PO_func, axis=1)
#PR<8.75
def PR_func(x):
    if (x['Staffed Hours']*24 + x['Exception request']*24)<8.75 :
        return 1
    else:         
        return 0
dailytracking['PR < 8.75']= dailytracking.apply(PR_func, axis=1)
#PR (MTD)
def PRMTD_func(x):
    if x['EID']=='' :
        return 0
    else:         
        return 1
dailytracking['PR count']= dailytracking.apply(PRMTD_func, axis=1)
dailytracking['PR (MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['PR count'].cumsum()
#Low PerF
def LowPerF_func(x):
    if x['CPH - Agent']<2 :
        return 1
    else:         
        return 0
dailytracking['Low PerF']= dailytracking.apply(LowPerF_func, axis=1)
#Login On Dayoff
def LoginonDayoff_func(x):
    if x['ATD MM Shift'] =='OFF' and x['ATD MM Ramco']=='OFF' and x['Staffed Hours']>0:
        return 1
    else:         
        return 0
dailytracking['Login on Dayoff']= dailytracking.apply(LoginonDayoff_func, axis=1)

#Not Logout
import re
def split1_it(startshift):
    return re.findall('(\d{4})-', startshift)
dailytracking['Shift1'] = dailytracking['Shift'].apply(split1_it)
dailytracking['Shift1']=dailytracking['Shift1'].astype(str)
dailytracking['Shift1'] = dailytracking['Shift1'] .str.replace("['", "")
dailytracking['Shift1'] =dailytracking['Shift1'] .str.replace("']", "")
dailytracking['Shift1'] =dailytracking['Shift1'] .str.replace("00", "")
dailytracking['Shift1'] =dailytracking['Shift1'] .str.replace("[]", '')
def split_it(endshift):
    return re.findall('\d{4}-(\d{4})', endshift)
dailytracking['Shift2'] = dailytracking['Shift'].apply(split_it)
dailytracking['Shift2']=dailytracking['Shift2'].astype(str)
dailytracking['Shift2'] = dailytracking['Shift2'] .str.replace("['", "")
dailytracking['Shift2'] =dailytracking['Shift2'] .str.replace("']", "")
dailytracking['Shift2'] =dailytracking['Shift2'] .str.replace("00", "")
dailytracking['Shift2'] =dailytracking['Shift2'] .str.replace("[]", '')
def shift1_func(x):
    if x['Shift1'] =='':
        return 0
    else:         
        return x['Shift1']
dailytracking['Shift1']= dailytracking.apply(shift1_func, axis=1)
dailytracking['Shift1']=dailytracking['Shift1'].astype('Float64')
dailytracking['Shift1']=dailytracking['Shift1'].astype('Int64')
def shift2_func(x):
    if x['Shift2'] =='':
        return 0
    else:         
        return x['Shift2']
dailytracking['Shift2']= dailytracking.apply(shift2_func, axis=1)
dailytracking['Shift2']=dailytracking['Shift2'].astype('Float64')
dailytracking['Shift2']=dailytracking['Shift2'].astype('Int64')
def time1_func(x):
    if x['Shift1'] =='':
        return t(0,0,0)
    elif x['Shift1'] ==24:
        return t(23,59,59)
    else:         
        return t(x['Shift1'],0,0)
dailytracking['Time1']= dailytracking.apply(time1_func, axis=1)
def time_func(x):
    if x['Shift2'] =='':
        return t(0,0,0)
    elif x['Shift2'] ==24:
        return t(23,59,59)
    else:         
        return t(x['Shift2'],0,0)
dailytracking['Time2']= dailytracking.apply(time_func, axis=1)
dailytracking['Time1'] = pd.to_datetime(dailytracking['Time1'], format='%H:%M:%S', errors='coerce').dt.time
dailytracking['Time2'] = pd.to_datetime(dailytracking['Time2'], format='%H:%M:%S', errors='coerce').dt.time
def nightshift_func(x):
    if x['Shift_type'] =="OFF":
        return x['Session Date']
    elif x['Shift2'] < 12:
        return x['Session Date']+ pd.Timedelta(days=1)
    else:         
        return x['Session Date']
dailytracking['Date']= dailytracking.apply(nightshift_func, axis=1) 
dailytracking['datetime1'] = pd.to_datetime(dailytracking['Session Date'].astype(str) + dailytracking['Time1'].astype(str), format = '%Y-%m-%d%H:%M:%S')
dailytracking['datetime2'] = pd.to_datetime(dailytracking['Date'].astype(str) + dailytracking['Time2'].astype(str), format = '%Y-%m-%d%H:%M:%S')
dailytracking['Approved Overtime1']=dailytracking['Approved Overtime'].astype('Float64')
dailytracking['Approved Overtime1']=dailytracking['Approved Overtime1'].apply(np.ceil)
dailytracking['datetime2'] = pd.to_datetime(dailytracking['datetime2'])
dailytracking['Approved Overtime1'] = pd.to_timedelta(dailytracking['Approved Overtime1'],'h')
dailytracking['datetime2'] = dailytracking['datetime2']+dailytracking['Approved Overtime1']
dailytracking['datetime2'] = dailytracking['datetime2']+ pd.Timedelta(minutes=5)
dailytracking['datetime1'] = pd.to_datetime(dailytracking['datetime1'])+ pd.Timedelta(minutes=5)
dailytracking['Logout Time']=dailytracking['Logout Time'].fillna(0)
dailytracking['datetime_logout'] = pd.to_datetime(dailytracking['Date'].astype(str) + dailytracking['Logout Time'].astype(str), format = '%Y-%m-%d%H:%M:%S', errors='coerce')
dailytracking['Login Time']=dailytracking['Login Time'].fillna(0)
dailytracking['datetime_login'] = pd.to_datetime(dailytracking['Session Date'].astype(str) + dailytracking['Login Time'].astype(str), format = '%Y-%m-%d%H:%M:%S', errors='coerce')
def notlogout_func(x):
    if x['Shift'] in {'AL','OFF','Training','HAL','CO','UPL'}:
        return ""
    elif x['datetime_logout'] > x['datetime2']:
        return "not logout after shift end"       
    else:         
        return ""
dailytracking['Not Logout']= dailytracking.apply(notlogout_func, axis=1)

# Late-Soon Login Logout
def late_func(x):
    if x['Shift'] in {'AL','OFF','Training','HAL','CO','UPL'}:
        return ""
    elif x['datetime_login'] > x['datetime1']:
        return "Late"       
    else:         
        return ""
dailytracking['Late']= dailytracking.apply(late_func, axis=1)
def soon_func(x):
    if x['Shift'] in {'AL','OFF','Training','HAL','CO','UPL'}:
        return ""
    elif x['datetime_logout'] < x['datetime2']- pd.Timedelta(minutes=5):
        return "Soon"       
    else:         
        return ""
dailytracking['Soon']= dailytracking.apply(soon_func, axis=1)
def latesoon_func(x):
    if x['Late'] =="" and x['Soon']=="":
        return ""
    elif x['Late']!="" and x['Soon']=="":
        return x['Late']       
    elif x['Late']=="" and x['Soon']!="":
        return x['Soon']   
    else:         
        return x['Late']+'-'+x['Soon']
dailytracking['Late-Soon']= dailytracking.apply(latesoon_func, axis=1)

# Overconsecutive
dailytracking= dailytracking.sort_values(by = ['EID', 'Session Date'])
dailytracking['level_1']=(dailytracking['Session Date'] - dailytracking['Session Date'].shift(1)).astype('int64')/86400000000000 #to get the discrepancy between current and previous visible working day
dailytracking['pre_ramco']=dailytracking['RAMCO'].shift(1)
def level1_func(x):
    if x['RAMCO'] in {'MTLX', 'MTLA', 'LWP', 'AL', 'AB', 'WO', 'SL', 'NM', 'MTL', 'CSLA', 'MOE', 'NCH', 'CSLB', 'PTL', 'PTL', 'PNL', 'CAL', 'HO', 'CDH', 'CO', 'CM', 'GM', 'MSL', 'VDH', 'MOC', 'CMLF'}:
        return 2
    elif x['level_1']==1 and x['pre_ramco'] in {'MTLX', 'MTLA', 'LWP', 'AL', 'AB', 'WO', 'SL', 'NM', 'MTL', 'CSLA', 'MOE', 'NCH', 'CSLB', 'PTL', 'PTL', 'PNL', 'CAL', 'HO', 'CDH', 'CO', 'CM', 'GM', 'MSL', 'VDH', 'MOC', 'CMLF'}:
        return 2
    else:         
        return x['level_1']
dailytracking['level_1']= dailytracking.apply(level1_func, axis=1)

dailytracking['check_eid'] = (dailytracking['EID'] - dailytracking['EID'].shift(1))

dailytracking[ 'level_2'] = ((dailytracking['level_1']!=1) | (dailytracking['check_eid'].isna()) | (dailytracking['check_eid']!=0)).cumsum()

dailytracking['OverConsecutive'] = (
    dailytracking[['level_2']]
    .assign(OverConsecutive=1)
    .groupby('level_2')['OverConsecutive']
    .transform('cumsum')
)
dailytracking['OverConsecutive'] = dailytracking.apply(lambda x: 'OverConsecutive' if x['OverConsecutive'] > 6 else x['OverConsecutive'], axis=1)
dailytracking['Staffed Hours']=dailytracking['Staffed Hours']*24
dailytracking['Exception request']=dailytracking['Exception request']/60

limit_date = max(pd.to_datetime(dailytracking['Session Date'], format ='mixed')) - pd.Timedelta(days=1)
dailytracking=dailytracking.loc[dailytracking['Session Date'] <= limit_date]

dailytracking['Session Date'] = pd.to_datetime(dailytracking['Session Date'], format='mixed').dt.date
dailytracking_raw = dailytracking.drop(columns=['ATD MM Shift','ATD MM Ramco','PO','PO Hour','PR count','level_1','pre_ramco','check_eid',
                                            'level_2','Shift1','Shift2','Time1','Time2','Date','datetime1','datetime2','Approved Overtime1',
                                            'datetime_logout','datetime_login','Late','Soon'])
dailytracking_raw['Session Date'] = pd.to_datetime(dailytracking_raw['Session Date'], format='mixed').dt.date
dailytracking_raw['Productive hours']=dailytracking_raw['Productive hours']*24
dailytracking_raw['Downtime']=dailytracking_raw['Downtime']*24
dailytracking_raw['Delivery hours']=dailytracking_raw['Delivery hours']*24
dailytracking_raw['Other codes']=dailytracking_raw['Other codes']*24
dailytracking_raw['New Hire Training']=dailytracking_raw['New Hire Training']*24
dailytracking_raw['Lunch Time']=dailytracking_raw['Lunch Time']*24
dailytracking_raw['Break Time']=dailytracking_raw['Break Time']*24
dailytracking_raw['Picklist - off Phone']=dailytracking_raw['Picklist - off Phone']*24
dailytracking_raw['Ready or Talking']=dailytracking_raw['Ready or Talking']*24
dailytracking_raw['Meeting']=dailytracking_raw['Meeting']*24
dailytracking_raw['Training']=dailytracking_raw['Training']*24
dailytracking_raw['Night Productive Hours']=dailytracking_raw['Night Productive Hours']*24
dailytracking_raw['Day Productive Hours']=dailytracking_raw['Day Productive Hours']*24
dailytracking_raw['Night Downtime Hours']=dailytracking_raw['Night Downtime Hours']*24
dailytracking_raw['Day Downtime Hours']=dailytracking_raw['Day Downtime Hours']*24
dailytracking_raw.replace([np.inf, -np.inf], 0, inplace=True)
dailytracking_full=dailytracking_raw
# dailytracking_full['Session Date'] = pd.to_datetime(dailytracking_full['Session Date'],format='mixed').dt.strftime('%Y-%m-%d')

In [17]:
## DEFECT
pivot=dailytracking[['Session Date','EID','Supervisor','ATD MM','#PO (MTD)','POHrs(MTD)','OT (MTD)','OT-LessDelivery',
                     'OT>4Hrs On PR \n OT>8Hrs On PO','PR < 8.75','Low PerF','Login on Dayoff','Not Logout','Late-Soon']]
pivot['Session Date']=pd.to_datetime(pivot['Session Date'],format='mixed')
pivot['Month']=pivot['Session Date'].dt.strftime('%Y-%m')
    #MM
atdmm=pivot['ATD MM']=='ATD MM'
pivot.loc[atdmm, 'MM'] = 1
pivot['MM'] = pivot['MM'].fillna(0)

    #Not Logout after shift end
notlogout=pivot['Not Logout']=='not logout after shift end'
pivot.loc[notlogout, 'Not Logout after shift end'] = 1
pivot['Not Logout after shift end'] = pivot['Not Logout after shift end'].fillna(0)

    #Late
late=pivot['Late-Soon']=='Late'
pivot.loc[late, 'Late'] = 1
pivot['Late'] = pivot['Late'].fillna(0)

    #Soon
soon=pivot['Late-Soon']=='Soon'
pivot.loc[soon, 'Soon'] = 1
pivot['Soon'] = pivot['Soon'].fillna(0)

    #Late
latesoon=pivot['Late-Soon']=='Late-Soon'
pivot.loc[latesoon, 'Late-Soon '] = 1
pivot['Late-Soon '] = pivot['Late-Soon '].fillna(0)

    #OT MTD > 40
ot40=pivot['OT (MTD)'] > 40
pivot.loc[ot40, 'OT > 40'] = 1
pivot['OT > 40'] = pivot['OT > 40'].fillna(0)
    
    #PO MTD >4
po4=pivot['#PO (MTD)'] > 4
pivot.loc[po4, 'PO > 4'] = 1
pivot['PO > 4'] = pivot['PO > 4'].fillna(0)

    #PO Hrs >32
pohrs32=pivot['POHrs(MTD)'] > 32
pivot.loc[pohrs32, 'PO hrs > 32'] = 1
pivot['PO hrs > 32'] = pivot['PO hrs > 32'].fillna(0)

pivot=pivot[['Supervisor','Month','MM','PR < 8.75','OT-LessDelivery','Low PerF','OT>4Hrs On PR \n OT>8Hrs On PO', 'Login on Dayoff',
             'Not Logout after shift end','Late','Soon','Late-Soon ','OT > 40','PO > 4','PO hrs > 32']]
pivot=pivot.groupby(['Supervisor','Month'], as_index=False).sum()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\2806513175.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot['Session Date']=pd.to_datetime(pivot['Session Date'],format='mixed')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\2806513175.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot['Month']=pivot['Session Date'].dt.strftime('%Y-%m')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\2806513175.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [18]:
# IMPORT BUDGET AND RATE
budget = pd.read_json(os.path.join(user_credential, r'DB\filejson\requirement.json'))
budget['Budget Downtime Hrs']=budget['Daily Requirement']-budget['Prod Requirement']
budget=budget.rename(columns={'Prod Requirement':'Budget Productive Hrs'})
rate = import_csv(path = budget_path)

In [19]:
#IMPORT ACTUAL INVOICE
actual = import_xlsx(path = budget_path,sheet_name = 'Sheet1')
actual=actual[['LOB','Month','Hours','Revenue']]
actual=actual.groupby(['LOB','Month'], as_index=False).sum()

actual_hour=actual.drop(columns=['Revenue'])
actual_hour_total=actual[['Month','Hours']]
actual_hour_total=actual_hour_total.groupby('Month', as_index=False).sum()
actual_hour_total['LOB']=actual_hour_total.apply(lambda x: 'Total Hour' if x['Month'] !='' else 0, axis=1)

actual_revenue=actual.drop(columns=['Hours'])
actual_revenue_total=actual[['Month','Revenue']]
actual_revenue_total=actual_revenue_total.groupby('Month', as_index=False).sum()
actual_revenue_total['LOB']=actual_revenue_total.apply(lambda x: 'Total Revenue' if x['Month'] !='' else 0, axis=1)

In [20]:
# ESTIMATE HOUR
max_date=dailytracking[['Session Date']]
max_date['Session Date']=pd.to_datetime(max_date['Session Date'],format='mixed')
max_date['Month']=max_date['Session Date'].dt.strftime('%Y-%m')
max_date['Day']=max_date['Session Date'].dt.day
max_date=max_date[['Month','Day','Session Date']]
max_date=max_date.groupby(['Month'], as_index=False).max()
max_date['EndOfMonth'] = pd.to_datetime(max_date['Session Date'], format="mixed") + MonthEnd(0)
max_date['EndDate']=max_date['EndOfMonth'].dt.day
max_date=max_date[['Month','Day','EndDate']]
estimate=dailytracking[['Session Date','LOB','RAMCO','Delivery hours','New Hire Training','Night Productive Hours',
               'Day Productive Hours','Night Downtime Hours','Day Downtime Hours','Approved Overtime']]
estimate['Session Date']=pd.to_datetime(estimate['Session Date'],format='mixed')
estimate['Month']=estimate['Session Date'].dt.strftime('%Y-%m')

not_holiday=estimate['RAMCO'] !='PH'
public_holiday=estimate['RAMCO'] =='PH'
estimate.loc[not_holiday, 'Productive Hours (Core)'] = estimate['Day Productive Hours']*24-estimate['Approved Overtime']
estimate['Productive Hours (Core)'] = estimate['Productive Hours (Core)'].fillna(0)

estimate.loc[not_holiday, 'Productive Hours (Night)'] = estimate['Night Productive Hours']*24
estimate['Productive Hours (Night)'] = estimate['Productive Hours (Night)'].fillna(0)

estimate.loc[not_holiday, 'Downtime Hours (Core)'] = estimate['Day Downtime Hours']*24
estimate['Downtime Hours (Core)'] = estimate['Downtime Hours (Core)'].fillna(0)

estimate.loc[not_holiday, 'Downtime Hours (Night)'] = estimate['Night Downtime Hours']*24
estimate['Downtime Hours (Night)'] = estimate['Downtime Hours (Night)'].fillna(0)

estimate.loc[not_holiday, 'New Hire Training Hours'] = estimate['New Hire Training']*24
estimate['New Hire Training Hours'] = estimate['New Hire Training Hours'].fillna(0)

estimate.loc[public_holiday, 'Public Holiday Hours'] =estimate['Delivery hours']*24
estimate['Public Holiday Hours'] = estimate['Public Holiday Hours'].fillna(0)

estimate_hour=estimate[['Month','LOB','Productive Hours (Core)','Productive Hours (Night)','Approved Overtime',
                        'Public Holiday Hours','Downtime Hours (Core)','Downtime Hours (Night)','New Hire Training Hours']]
estimate_hour=estimate_hour.groupby(['Month','LOB'], as_index=False).sum()
estimate_hour['Total Hours']=estimate_hour['Productive Hours (Core)']+estimate_hour['Productive Hours (Night)']+estimate_hour['Approved Overtime']+estimate_hour['Downtime Hours (Core)']+estimate_hour['Downtime Hours (Night)']+estimate_hour['New Hire Training Hours']+estimate_hour['Public Holiday Hours']
estimate_hour=pd.merge(estimate_hour,max_date,left_on='Month',right_on='Month',how='left')
estimate_hour['Forecast to EOM']=(estimate_hour['Total Hours']/estimate_hour['Day'])*estimate_hour['EndDate']
estimate_hour=estimate_hour.drop(columns=['Day', 'EndDate'])
estimate_hour=estimate_hour.rename(columns={'Approved Overtime':'OT Hours'})
actual_hour=actual[['LOB','Month','Hours']]
estimate_hour=pd.merge(estimate_hour,actual_hour,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')

# ESTIMATE REVENUE
estimate_revenue=estimate[['Month','LOB','Productive Hours (Core)','Productive Hours (Night)','Approved Overtime',
                                           'Public Holiday Hours','Downtime Hours (Core)','Downtime Hours (Night)',
                                           'New Hire Training Hours']]
estimate_revenue=pd.merge(estimate_revenue,rate,left_on='Month',right_on='Month',how='left')
estimate_revenue['Productive Hours (Core)'] = estimate_revenue['Productive Hours (Core)']*estimate_revenue['Conversion']*estimate_revenue['Productive hours (Core)']
estimate_revenue['Productive Hours (Night)'] = estimate_revenue['Productive Hours (Night)']*estimate_revenue['Conversion']*estimate_revenue['Productive hours (Night)']
estimate_revenue['Downtime Hours (Core)'] = estimate_revenue['Downtime Hours (Core)']*estimate_revenue['Conversion']*estimate_revenue['Downtime (Core)']
estimate_revenue['Downtime Hours (Night)'] = estimate_revenue['Downtime Hours (Night)']*estimate_revenue['Conversion']*estimate_revenue['Downtime (Night)']
estimate_revenue['New Hire Training Hours'] = estimate_revenue['New Hire Training Hours']*estimate_revenue['Conversion']*estimate_revenue['New hire']
estimate_revenue['Public Holiday Hours'] = estimate_revenue['Public Holiday Hours']*estimate_revenue['Conversion']*estimate_revenue['PH']
estimate_revenue['Approved Overtime'] = estimate_revenue['Approved Overtime']*estimate_revenue['Conversion']*estimate_revenue['OT']

estimate_revenue=estimate_revenue.groupby(['Month','LOB'], as_index=False).sum()

estimate_revenue['Total Revenue']=estimate_revenue['Productive Hours (Core)']+estimate_revenue['Productive Hours (Night)']+estimate_revenue['Approved Overtime']+estimate_revenue['Downtime Hours (Core)']+estimate_revenue['Downtime Hours (Night)']+estimate_revenue['New Hire Training Hours']+estimate_revenue['Public Holiday Hours']

estimate_revenue=pd.merge(estimate_revenue,max_date,left_on='Month',right_on='Month',how='left')

estimate_revenue['Forecast to EOM']=(estimate_revenue['Total Revenue']/estimate_revenue['Day'])*estimate_revenue['EndDate']

estimate_revenue=estimate_revenue.drop(columns=['Day', 'EndDate'])
estimate_revenue=estimate_revenue.rename(columns={'Approved Overtime':'OT Revenue','Productive Hours (Core)':'Productive Revenue (Core)',
                                                 'Productive Hours (Night)':'Productive Revenue (Night)',
                                                 'Public Holiday Hours':'Public Holiday Revenue','Downtime Hours (Core)':'Downtime Revenue (Core)',
                                                 'Downtime Hours (Night)':'Downtime Revenue (Night)','New Hire Training Hours':'New Hire Training Revenue'})
actual_revenue=actual[['LOB','Month','Revenue']]
estimate_revenue=pd.merge(estimate_revenue,actual_revenue,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\1967466067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_date['Session Date']=pd.to_datetime(max_date['Session Date'],format='mixed')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\1967466067.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_date['Month']=max_date['Session Date'].dt.strftime('%Y-%m')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\1967466067.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [21]:
# BUDGET HOUR
budget_hour=budget[['Date','LOB','Budget Productive Hrs','Budget Downtime Hrs']]
budget_hour['Month']=budget_hour['Date'].dt.strftime('%Y-%m')
budget_hour['Budget Hours']=budget_hour['Budget Productive Hrs']+budget_hour['Budget Downtime Hrs']
budget_hour=budget_hour.drop(columns=['Date','Budget Productive Hrs','Budget Downtime Hrs'])
budget_hour=budget_hour.groupby(['Month','LOB'], as_index=False).sum()

# BUDGET REVENUE
budget_revenue=budget[['Date','LOB','Budget Productive Hrs','Budget Downtime Hrs']]
budget_revenue['Month']=budget_revenue['Date'].dt.strftime('%Y-%m')
budget_revenue=budget_revenue.drop(columns=['Date'])
budget_revenue=budget_revenue.groupby(['Month','LOB'], as_index=False).sum()

estimate_hour_budget=estimate_hour
estimate_hour_budget['% Productive Night']=(estimate_hour_budget['Productive Hours (Night)']/estimate_hour_budget['Productive Hours (Core)'])*0.7
estimate_hour_budget['% Downtime Night']=(estimate_hour_budget['Downtime Hours (Night)']/estimate_hour_budget['Downtime Hours (Core)'])*0.7
estimate_hour_budget=estimate_hour_budget[['Month','LOB','% Productive Night','% Downtime Night','OT Hours','Public Holiday Hours']]

budget_revenue=pd.merge(budget_revenue,estimate_hour_budget,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')
budget_revenue['Budget Productive Hrs']=budget_revenue['Budget Productive Hrs']-(budget_revenue['OT Hours']*0.95)
budget_revenue['Budget Downtime Hrs']=budget_revenue['Budget Downtime Hrs']-(budget_revenue['OT Hours']*0.05)
budget_revenue['Budget Productive Hrs']=budget_revenue['Budget Productive Hrs']-(budget_revenue['Public Holiday Hours']*0.95)
budget_revenue['Budget Downtime Hrs']=budget_revenue['Budget Downtime Hrs']-(budget_revenue['Public Holiday Hours']*0.05)

budget_revenue['Productive Hours (Night)']=budget_revenue['Budget Productive Hrs']*budget_revenue['% Productive Night']
budget_revenue['Productive Hours (Core)']=budget_revenue['Budget Productive Hrs']-budget_revenue['Productive Hours (Night)']
budget_revenue['Downtime Hours (Night)']=budget_revenue['Budget Downtime Hrs']*budget_revenue['% Downtime Night']
budget_revenue['Downtime Hours (Core)']=budget_revenue['Budget Downtime Hrs']-budget_revenue['Downtime Hours (Night)']
budget_revenue=pd.merge(budget_revenue,rate,left_on='Month',right_on='Month',how='left')
budget_revenue['Productive Revenue (Core)']=budget_revenue['Productive Hours (Core)']*budget_revenue['Productive hours (Core)']
budget_revenue['Productive Revenue (Night)']=budget_revenue['Productive Hours (Night)']*budget_revenue['Productive hours (Night)']
budget_revenue['Downtime Revenue (Core)']=budget_revenue['Downtime Hours (Core)']*budget_revenue['Downtime (Core)']
budget_revenue['Downtime Revenue (Night)']=budget_revenue['Downtime Hours (Night)']*budget_revenue['Downtime (Night)']
budget_revenue['OT Revenue']=budget_revenue['OT Hours']*budget_revenue['OT']
budget_revenue['PH Revenue']=budget_revenue['Public Holiday Hours']*budget_revenue['PH']
budget_revenue['Budget Revenue']=budget_revenue['Productive Revenue (Core)']+budget_revenue['Productive Revenue (Night)']+budget_revenue['Downtime Revenue (Core)']+budget_revenue['Downtime Revenue (Night)']+budget_revenue['OT Revenue']+budget_revenue['PH Revenue']
budget_revenue=budget_revenue[['Month','LOB','Budget Revenue']]

# FINAL ESTIMATION
estimate_hour=pd.merge(estimate_hour,budget_hour,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')
estimate_hour=estimate_hour[['Month','LOB','Productive Hours (Core)','Productive Hours (Night)','OT Hours','Public Holiday Hours','Downtime Hours (Core)','Downtime Hours (Night)','New Hire Training Hours','Budget Hours','Total Hours','Forecast to EOM']]
estimate_hour=pd.merge(estimate_hour,actual_hour,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')
estimate_hour_total=estimate_hour.drop(columns=['LOB'])
estimate_hour_total=estimate_hour_total.groupby(['Month'], as_index=False).sum()
estimate_hour_total['LOB']=estimate_hour_total.apply(lambda x: 'Total Hour' if x['Month'] !='' else 0, axis=1)
estimate_hour_total=pd.merge(estimate_hour_total,actual_hour_total,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')
estimate_hour_total=estimate_hour_total.rename(columns={'Hours_x':'Hours'})
estimate_hour =pd.concat([estimate_hour, estimate_hour_total], sort=False)
estimate_hour=estimate_hour.drop(columns=['Hours_y'])

estimate_revenue=pd.merge(estimate_revenue,budget_revenue,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')
estimate_revenue=estimate_revenue[['Month','LOB','Productive Revenue (Core)','Productive Revenue (Night)','OT Revenue','Public Holiday Revenue','Downtime Revenue (Core)','Downtime Revenue (Night)','New Hire Training Revenue','Budget Revenue','Total Revenue','Forecast to EOM']]
estimate_revenue=pd.merge(estimate_revenue,actual_revenue,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')
estimate_revenue_total=estimate_revenue.drop(columns=['LOB'])
estimate_revenue_total=estimate_revenue_total.groupby(['Month'], as_index=False).sum()
estimate_revenue_total['LOB']=estimate_revenue_total.apply(lambda x: 'Total Revenue' if x['Month'] !='' else 0, axis=1)
estimate_revenue_total=pd.merge(estimate_revenue_total,actual_revenue_total,left_on=['Month','LOB'],right_on=['Month','LOB'],how='left')
estimate_revenue_total=estimate_revenue_total.rename(columns={'Revenue_x':'Revenue'})
estimate_revenue =pd.concat([estimate_revenue, estimate_revenue_total], sort=False)
estimate_revenue=estimate_revenue.drop(columns=['Revenue_y'])


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\1200638067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget_hour['Month']=budget_hour['Date'].dt.strftime('%Y-%m')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\1200638067.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget_revenue['Month']=budget_revenue['Date'].dt.strftime('%Y-%m')


In [22]:
# Format Daily Tracking
import xlsxwriter
import openpyxl
writer = pd.ExcelWriter(dailytracking_path, engine="xlsxwriter")
dailytracking_raw.to_excel(writer, sheet_name="Daily Tracking", startrow=0, index=False)
pivot.to_excel(writer, sheet_name="DEFECT", startrow=0, index=False)
workbook = writer.book
dailytracking_sheet= writer.sheets["Daily Tracking"]
defect = writer.sheets["DEFECT"]

#Add format for Daily Tracking sheet
dailytracking_sheet.autofilter(0, 0, dailytracking_raw.shape[0], dailytracking_raw.shape[1]-1)
dailytracking_sheet.freeze_panes(1, 0)
# Add format for DEFECT sheet
header_defect = workbook.add_format(
    {
        "bold": True,
        "text_wrap": True,
        "valign": "top",
        "fg_color": "#4472C4",
        "border": 0,
        'font_color': 'white'
    }
)
for col_num, value in enumerate(pivot.columns.values):
    defect.write(0, col_num, value, header_defect)
defect.autofilter(0, 0, pivot.shape[0], pivot.shape[1]-1)
defect.freeze_panes(1, 0)
defect.set_column('A:O', 11)

writer.close()

In [23]:
# Format Revenue Estimation
import xlsxwriter
import openpyxl
writer = pd.ExcelWriter(revenue_path, engine="xlsxwriter")
dailytracking_raw.to_excel(writer, sheet_name="DailyTracking", startrow=0, index=False)
pivot.to_excel(writer, sheet_name="DEFECT", startrow=0, index=False)
estimate_hour.to_excel(writer, sheet_name="ESTIMATE_HOUR", startrow=0, index=False)
estimate_revenue.to_excel(writer, sheet_name="ESTIMATE_REVENUE", startrow=0, index=False)
rate.to_excel(writer, sheet_name="Rate", startrow=0, index=False)
define.to_excel(writer, sheet_name="DEFINE", startrow=0, index=False)

workbook = writer.book
dailytracking_sheet= writer.sheets["DailyTracking"]
defect = writer.sheets["DEFECT"]
estimatehour_sheet=writer.sheets["ESTIMATE_HOUR"]
estimaterevenue_sheet=writer.sheets["ESTIMATE_REVENUE"]
estimation=workbook.add_worksheet("ESTIMATION")
rate=writer.sheets["Rate"]
define=writer.sheets["DEFINE"]
#Add format for Daily Tracking sheet
dailytracking_sheet.autofilter(0, 0, dailytracking_raw.shape[0], dailytracking_raw.shape[1]-1)
dailytracking_sheet.freeze_panes(1, 0)
# Add format for DEFECT sheet
header_defect = workbook.add_format(
    {
        "bold": True,
        "text_wrap": True,
        "valign": "top",
        "fg_color": "#4472C4",
        "border": 0,
        'font_color': 'white'
    }
)
for col_num, value in enumerate(pivot.columns.values):
    defect.write(0, col_num, value, header_defect)
defect.autofilter(0, 0, pivot.shape[0], pivot.shape[1]-1)
defect.freeze_panes(1, 0)
defect.set_column('A:O', 11)

#Hide estimate hour and estimate revenue sheet
dailytracking_sheet.hide()
estimatehour_sheet.hide()
estimaterevenue_sheet.hide()
defect.hide()
rate.hide()
# Add value for ESTIMATION Sheet
first_format = workbook.add_format({"bold": True,"text_wrap": True,"valign": "middle","fg_color": "#833C0C","border": 2,'font_color': 'white'})
second_format = workbook.add_format({"bold": True,"text_wrap": True,"valign": "middle","fg_color": "#161616","border": 2,'font_color': 'white','num_format': '$#,##0.##'})
third_format = workbook.add_format({"bold": True,"text_wrap": True,"valign": "middle","align":"center","fg_color": "#375623","border": 2,'font_color': 'white'})
third_format.set_align('vcenter')

thisyear= (pd.Timestamp.today().date()).year
thismonth= (pd.Timestamp.today().date()).month                        
estimation.write('A1', 'YEAR',first_format)
estimation.write('B1', thisyear,first_format)
estimation.write('D1', 'MONTH',first_format)
estimation.write('E1', thismonth,first_format)
estimation.write('G1', 'Viewed by',first_format)
estimation.data_validation('H1', {'validate': 'list',
                                  'source': ['Revenue ($)', 'Hour']}) 
estimation.write('H1', 'Revenue ($)',first_format)

estimation.write('A3', 'Rate',second_format)
estimation.write_formula('B3', '=XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!$B:$B)*XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!C:C)',second_format)
estimation.write_formula('C3', '=XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!$B:$B)*XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!D:D)',second_format)
estimation.write_formula('D3', '=XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!$B:$B)*XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!E:E)',second_format)
estimation.write_formula('E3', '=XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!$B:$B)*XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!F:F)',second_format)
estimation.write_formula('F3', '=XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!$B:$B)*XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!G:G)',second_format)
estimation.write_formula('G3', '=XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!$B:$B)*XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!H:H)',second_format)
estimation.write_formula('H3', '=XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!$B:$B)*XLOOKUP($B1&"-"&$E1,Rate!$A:$A,Rate!I:I)',second_format) 
estimation.write('I3', '',second_format)
estimation.write('J3', '',second_format)
estimation.write('K3', '',second_format)
estimation.write('L3', '',second_format)
                                  
estimation.write('A4', 'LOB',third_format)
estimation.write('B4', 'Producted (Core)',third_format)
estimation.write('C4', 'Producted (Night)',third_format)
estimation.write('D4', 'OT',third_format)
estimation.write('E4', 'Public Holiday',third_format)
estimation.write('F4', 'Downtime (Core)',third_format)
estimation.write('G4', 'Downtime (Night)',third_format)
estimation.write('H4', 'New Hire Training',third_format)
estimation.write('I4', 'Budget to EOM',third_format)
estimation.write_formula('J4', '="Total to "&CHAR(10)&TEXT(MAXIFS(DailyTracking!B:B,DailyTracking!B:B,"<="&EOMONTH(DATE(B1,E1,1),0),DailyTracking!B:B,">="&DATE(ESTIMATION!B1,ESTIMATION!E1,1)),"dd-mmm")',third_format)
estimation.write('K4', 'Forecast to EOM',third_format)
estimation.write('L4', 'Actual Invoice',third_format)

estimation.write_formula('A5', '=IFS(H1="Hour",FILTER(ESTIMATE_HOUR!B2:M1000000,ESTIMATE_REVENUE!A2:A1000000=B1&"-"&E1),H1="Revenue ($)",FILTER(ESTIMATE_REVENUE!B2:M1000000,ESTIMATE_REVENUE!A2:A1000000=B1&"-"&E1))')

first_row=4
last_row=99
first_col=0
last_col=11
currency_format = workbook.add_format({'num_format': '$#,##0'})
number_format = workbook.add_format({'num_format': '#,##0'})
total_format= workbook.add_format({"bold": True,"text_wrap": True,"valign": "middle","align":"center","border": 0,'font_color': 'black'})
estimation.conditional_format(first_row, first_col, last_row, last_col,
                             {'type': 'formula','criteria': '=$H$1="Revenue ($)"','format': currency_format})
estimation.conditional_format(first_row, first_col, last_row, last_col,
                             {'type': 'formula','criteria': '=$H$1="Hour"','format': number_format})
estimation.conditional_format(first_row, first_col, last_row, last_col,
                             {'type': 'formula','criteria': '=$A5:$A100="Total Hour"','format': total_format})
estimation.conditional_format(first_row, first_col, last_row, last_col,
                             {'type': 'formula','criteria': '=$A5:$A100="Total Revenue"','format': total_format})
estimation.set_column('A:K', 13.86)
writer.close()

In [24]:
# ADHERENCE
workplan = pd.read_json(os.path.join(user_credential, r'DB\filejson\workplan_merge_iex.json'))
workplan=workplan[['Date','Agent ID','Scheduled Activity','Length','EID']]
workplan['Date']=pd.to_datetime(workplan['Date']).dt.date
workplan['EID']=workplan['EID'].astype("Int64")

workplan['Plotted Downtime']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] in {'Coaching 1:1','Team Meeting'} else 0, axis=1)
workplan['Plotted Phone']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity']=='Open Time' else 0, axis=1)
workplan['Plotted Picklist']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] =='Email 1' else 0, axis=1)
workplan['Plotted Lunch']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] =='Lunch' else 0, axis=1)
workplan['Plotted Break']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] =='Break Offline' else 0, axis=1)

workplan=workplan.groupby(['Date','Agent ID','EID'], as_index=False).sum()
workplan['Plotted Productive Hour']=workplan['Plotted Phone']+workplan['Plotted Picklist']

eps_adherence=dailytracking[['Session Date','EID','Full name','Booking Login ID','LOB','Supervisor','Wave #',
                                 'Picklist - off Phone','Ready or Talking','Meeting','Training','Lunch Time','Break Time']]
eps_adherence['Actual Downtime']=eps_adherence['Meeting']+eps_adherence['Training']
eps_adherence['Session Date']=pd.to_datetime(eps_adherence['Session Date']).dt.date
adherence=pd.merge(eps_adherence,workplan,left_on=['Session Date','EID'],right_on=['Date','EID'],how='left')
adherence['Actual Productive Hour']=adherence['Picklist - off Phone']+adherence['Ready or Talking']
adherence=adherence.rename(columns={'Agent ID':'IEX ID','Picklist - off Phone':'Actual Picklist','Ready or Talking':'Actual Phone',
                                    'Lunch Time':'Actual Lunch','Break Time':'Actual Break'})
adherence=adherence[['Session Date','EID','Full name','IEX ID','Booking Login ID','LOB','Supervisor','Wave #',
                     'Plotted Downtime','Actual Downtime','Plotted Productive Hour','Actual Productive Hour',
                     'Plotted Phone','Actual Phone','Plotted Picklist','Actual Picklist',
                     'Plotted Lunch','Actual Lunch','Plotted Break','Actual Break']]
def downtimedeliveryper(x):
    if x['Plotted Downtime']==0 and x['Actual Downtime']!=0:
        return 0
    elif x['Plotted Downtime']==0 and x['Actual Downtime']==0:
        return 1
    else:         
        return x['Actual Downtime']/x['Plotted Downtime']
adherence['%Downtime Delivery']= adherence.apply(downtimedeliveryper, axis=1)

def producteddeliveryper(x):
    if x['Plotted Productive Hour']==0 and x['Actual Productive Hour']!=0:
        return 0
    elif x['Plotted Productive Hour']==0 and x['Actual Productive Hour']==0:
        return 1
    else:         
        return x['Actual Productive Hour']/x['Plotted Productive Hour']
adherence['%Productive Delivery']= adherence.apply(producteddeliveryper, axis=1)
adherence['Lunch variance']=(adherence['Actual Lunch']-adherence['Plotted Lunch'])/adherence['Plotted Lunch']
adherence['Break variance']=(adherence['Actual Break']-adherence['Plotted Break'])/adherence['Plotted Break']
adherence=adherence[['Session Date','EID','Full name','IEX ID','Booking Login ID','LOB','Supervisor','Wave #',
                     'Plotted Downtime','Actual Downtime','%Downtime Delivery','Plotted Productive Hour','Actual Productive Hour',
                     '%Productive Delivery','Plotted Phone','Actual Phone','Plotted Picklist','Actual Picklist',
                     'Plotted Lunch','Actual Lunch','Lunch variance','Plotted Break','Actual Break','Break variance']]
adherence.replace([np.inf, -np.inf], 0, inplace=True)
# export to a sheet in Adherence workbook
with pd.ExcelWriter(adherence_path, mode="a", engine="openpyxl",if_sheet_exists="replace") as writer1:
    adherence.to_excel(writer1, sheet_name="process",index=False)
# adherence.to_excel(r"C:\Users\thibaotram.nguyen\Concentrix Corporation\Dinh Hoang Nguyen - WFM-internal\Dispatch files\Adherence & Daily Tracking\adherence_test.xlsx",index=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\4278659990.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_adherence['Actual Downtime']=eps_adherence['Meeting']+eps_adherence['Training']
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_23096\4278659990.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_adherence['Session Date']=pd.to_datetime(eps_adherence['Session Date']).dt.date
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarn